# --------------------------------------------------------------------------------------------------------------
# PLAYAS de ESPAÑA 2017 : MongoDB
# --------------------------------------------------------------------------------------------------------------

## Inicializaciones previas

### a) Carga de librerías

In [1]:
import pymongo 
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

from pprintpp import pprint as pp

import pandas as pd
import datetime
import json

# A) Carga de datos desde Excel a un DataFrame de PANDAS

Leemos los datos desde el fichero Excel y los desnormalizamos : leemos las 2 hojas del fichero Excel en un **mismo** DataFrame de Pandas para guardarlo todo junto.

In [2]:
df_excel_provincias = pd.read_excel("./data/playas_2017.xlsx", sheetname= "HOJA_PROVINCIAS")
df_excel_playas     = pd.read_excel("./data/playas_2017.xlsx", sheetname= "HOJA_PLAYAS")

In [3]:
df_mongoDB = pd.merge(df_excel_provincias, df_excel_playas, on = ['id_provincia'], how = 'inner')
df_mongoDB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3511 entries, 0 to 3510
Data columns (total 12 columns):
id_provincia       3511 non-null int64
nom_provincia      3511 non-null object
comunidad          3511 non-null object
cod_playa          3511 non-null int64
nom_playa          3511 non-null object
localidad          3511 non-null object
clave_nom_playa    3511 non-null object
longitud           3511 non-null int64
longitud_txt       3511 non-null object
arena              3511 non-null object
nudista            3511 non-null object
bandera_azul       3511 non-null object
dtypes: int64(3), object(9)
memory usage: 356.6+ KB


# B) BD de MongoDB

## B.1) Conexión a la BD de MongoDB

### B.1.1) Conexión

In [4]:
# connect to database
connection_MDB = MongoClient('localhost', 27017)

In [5]:
!mongod --version

db version v3.4.2
git version: 3f76e40c105fc223b3e5aac3e20dcd026b83b38b
OpenSSL version: OpenSSL 1.0.1f 6 Jan 2014
allocator: tcmalloc
modules: none
build environment:
    distmod: ubuntu1404
    distarch: x86_64
    target_arch: x86_64


### B.1.2) Borrado de la BD ("playas_2017")

In [6]:
# Borramos la base de datos
connection_MDB.drop_database('playas_2017')

### B.1.3) Creación de la BD ("playas_2017")

In [7]:
# Pymongo permite una sintaxis practicamente igual que la original
db_MDB = connection_MDB.playas_2017

## B.2) Creamos las COLECCIONES de MongoDB

Las **colecciones** se crean en el mismo momento de insertar los datos.

# C) Carga de datos en MongoDB

## C.1) Transformaciones previas

Gracias a Pandas convertimos el dataset en formato JSON, que es el que vamos a insertar en MongoDB prácticamente sin cambios.

In [8]:
json_string = df_mongoDB.to_json(orient = 'records')
json_list = json.loads(json_string)

In [9]:
pp(json_list[0])

{
    u'arena': u'Dorada',
    u'bandera_azul': u'No',
    u'clave_nom_playa': u'Puerto del Rosario (Janubio)',
    u'cod_playa': 3001,
    u'comunidad': u'Canarias',
    u'id_provincia': 16,
    u'localidad': u'Puerto del Rosario',
    u'longitud': 230,
    u'longitud_txt': u'230 metros',
    u'nom_playa': u'Janubio',
    u'nom_provincia': u'Las Palmas',
    u'nudista': u'Sí',
}


En MongoDB vamos a crear una única **colección** (la colección **"playas"**) donde vamos a guardar todos los datos desnormalizados:

<br><br> 

<img src="images/Modelo_BD___MongoDB.JPG",width=150,height=50>

<br><br>

## C.2) Inserción de los datos en MongoDB

En la inserción de la información en MongoDB se podrían realizar algunas **tareas de limpieza** tales como :

   + **Convertir fechas** a un formato DateTime, de forma que mongo inserte el dato correctamente (en nuestra BD de playas no hay fechas)
   + **Eliminar campos** que no queremos tener en el modelo (no eliminamos ninguno por compararlo con el resto de BD)
   + **Eliminar campos nulos** (en nuestra BD de playas no hay campos nulos)

In [10]:
for playa_json in json_list:
    
    # Convertir fechas
    # playa_json['fecha'] = datetime.datetime.fromtimestamp(playa_json['fecha'] / 1e3)
    
    # Eliminar campos
    # del playa_json['arena']

    # Eliminar campos nulos
    # if  playa_json['comercio'] == None:
    #     del playa_json['comercio']
    
    db_MDB.playas.insert_one(playa_json)   # Creamos en MongoDB la colección "playas"

# ------------------------------------------------------------------------------
# QUERYS   (MongoDB)
# ------------------------------------------------------------------------------

Todas las querys de este caso de uso se pueden resolver directamente por la base de datos ...

### QUERY_1) Las 10 playas más largas (incluida la provincia)

In [11]:
rs = db_MDB.playas.find({},
{
    "id_provincia"    : 1,
    "nom_provincia"   : 1,
    "localidad"       : 1,
    "nom_playa"       : 1,
    "clave_nom_playa" : 1,
    "longitud"        : 1,
    "longitud_txt"    : 1,
    "_id"             : 0
}).limit(10).sort([("longitud", -1)])

In [12]:
type(rs)

pymongo.cursor.Cursor

In [13]:
pd.DataFrame(list(rs))

,clave_nom_playa,id_provincia,localidad,longitud,longitud_txt,nom_playa,nom_provincia
0,Almonte (Doñana),14,Almonte,28000,28.000 metros,Doñana,Huelva
1,Almonte (Castilla),14,Almonte,17000,17.000 metros,Castilla,Huelva
2,Pájara (El Cofete),16,Pájara,13700,13.700 metros,El Cofete,Las Palmas
3,Lepe (Nueva Umbría),14,Lepe,12000,12.000 metros,Nueva Umbría,Huelva
4,Palos de la Frontera (Mazagón),14,Palos de la Frontera,9000,9.000 metros,Mazagón,Huelva
5,El Ejido (Punta Entinas Sabinar),3,El Ejido,6500,6.500 metros,Punta Entinas Sabinar,Almería
6,Deltebre (Punta del Fangar),23,Deltebre,6500,6.500 metros,Punta del Fangar,Tarragona
7,Sant Pere Pescador (Sant Pere Pescador),12,Sant Pere Pescador,6400,6400 metros,Sant Pere Pescador,Girona
8,Sant Carles de la Ràpita (El Trabucador),23,Sant Carles de la Ràpita,6165,6.165 metros,El Trabucador,Tarragona
9,Carnota (Carnota),1,Carnota,5600,5.600 metros,Carnota,A Coruña


### QUERY_2) Las 10 playas más largas de una cierta provincia (incluído el nombre de la provincia)

In [14]:
rs = db_MDB.playas.find({"nom_provincia" : "A Coruña"},
{
    "id_provincia"    : 1,
    "nom_provincia"   : 1,
    "localidad"       : 1,
    "nom_playa"       : 1,
    "clave_nom_playa" : 1,
    "longitud"        : 1,
    "longitud_txt"    : 1,
    "_id"             : 0
}).limit(10).sort([("longitud", -1)])

type(rs)

pymongo.cursor.Cursor

In [15]:
pd.DataFrame(list(rs))

,clave_nom_playa,id_provincia,localidad,longitud,longitud_txt,nom_playa,nom_provincia
0,Carnota (Carnota),1,Carnota,5600,5.600 metros,Carnota,A Coruña
1,Carballo (Baldaio),1,Carballo,4000,4.000 metros,Baldaio,A Coruña
2,Valdoviño (A Frouxeira),1,Valdoviño,3000,3.000 metros,A Frouxeira,A Coruña
3,Ribeira (Vilar),1,Ribeira,2900,2.900 metros,Vilar,A Coruña
4,Ortigueira (Morouzos),1,Ortigueira,2800,2.800 metros,Morouzos,A Coruña
5,Laxe (Traba),1,Laxe,2650,2.650 metros,Traba,A Coruña
6,Boiro (Barraña),1,Boiro,2150,2.150 metros,Barraña,A Coruña
7,Porto do Son (Areas Longas),1,Porto do Son,2100,2.100 metros,Areas Longas,A Coruña
8,Boiro (Carragueiros),1,Boiro,2000,2.000 metros,Carragueiros,A Coruña
9,Fisterra (Langosteira),1,Fisterra,1970,1.970 metros,Langosteira,A Coruña


### QUERY_3) Las playas de una comunidad autónoma concreta (ordenadas por longitud)

In [16]:
rs = db_MDB.playas.find({"comunidad" : "Galicia"},
{
    "id_provincia"    : 1,
    "comunidad"       : 1,
    "nom_provincia"   : 1,
    "localidad"       : 1,
    "nom_playa"       : 1,
    "clave_nom_playa" : 1,
    "longitud"        : 1,
    "longitud_txt"    : 1,
    "_id"             : 0
}).limit(10).sort([("longitud", -1)])

type(rs)

pymongo.cursor.Cursor

In [17]:
pd.DataFrame(list(rs))

,clave_nom_playa,comunidad,id_provincia,localidad,longitud,longitud_txt,nom_playa,nom_provincia
0,Carnota (Carnota),Galicia,1,Carnota,5600,5.600 metros,Carnota,A Coruña
1,Carballo (Baldaio),Galicia,1,Carballo,4000,4.000 metros,Baldaio,A Coruña
2,Valdoviño (A Frouxeira),Galicia,1,Valdoviño,3000,3.000 metros,A Frouxeira,A Coruña
3,Ribeira (Vilar),Galicia,1,Ribeira,2900,2.900 metros,Vilar,A Coruña
4,Ortigueira (Morouzos),Galicia,1,Ortigueira,2800,2.800 metros,Morouzos,A Coruña
5,Laxe (Traba),Galicia,1,Laxe,2650,2.650 metros,Traba,A Coruña
6,O Grove (A Lanzada),Galicia,21,O Grove,2400,2.400 metros,A Lanzada,Pontevedra
7,Redondela (Da Punta),Galicia,21,Redondela,2400,2.400 metros,Da Punta,Pontevedra
8,Boiro (Barraña),Galicia,1,Boiro,2150,2.150 metros,Barraña,A Coruña
9,Porto do Son (Areas Longas),Galicia,1,Porto do Son,2100,2.100 metros,Areas Longas,A Coruña


### QUERY_4) Las 10 provincias que más metros de playa tienen

En este caso, al obtener un dato agregado, necesitamos utilizar la función **aggregate()** de mongo

#### 4.1) FORMATO_1 

In [18]:
rs = db_MDB.playas.aggregate([
        {"$group" : { "_id" : "$nom_provincia", 
                      "metros_de_playa" : { "$sum" : "$longitud"}}},        
        {"$sort" : {"metros_de_playa" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id"    : 0, 
                       "nom_provincia"   : "$_id", 
                       "metros_de_playa" : 1}
        }
    ])

type(rs)

pymongo.command_cursor.CommandCursor

In [19]:
df = pd.DataFrame(list(rs))
df

,metros_de_playa,nom_provincia
0,157040,A Coruña
1,138473,Cádiz
2,137337,Las Palmas
3,136720,Málaga
4,120765,Almería
5,118148,Alicante/Alacant
6,117767,Illes Balears
7,113720,Huelva
8,102431,Murcia
9,99015,Tarragona


#### 4.1) FORMATO_2 

In [20]:
rs = db_MDB.playas.aggregate([
        {"$group" : { "_id" : "$nom_provincia", 
                      "longitud" : { "$sum" : "$longitud"}}},        
        {"$sort" : {"longitud" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id"    : 0, 
                       "nom_provincia"   : "$_id", 
                       "metros_de_playa" : "$longitud"}
        }
    ])

type(rs)

pymongo.command_cursor.CommandCursor

In [21]:
df = pd.DataFrame(list(rs))
df

,metros_de_playa,nom_provincia
0,157040,A Coruña
1,138473,Cádiz
2,137337,Las Palmas
3,136720,Málaga
4,120765,Almería
5,118148,Alicante/Alacant
6,117767,Illes Balears
7,113720,Huelva
8,102431,Murcia
9,99015,Tarragona


### QUERY_5) Metros de playa de una cierta Comunidad Autónoma agrupados por provincia

In [22]:
rs = db_MDB.playas.aggregate([
        {"$match"   : {"comunidad": "Galicia"}},
        {"$group" : { "_id" : {"comunidad"     : "$comunidad", 
                               "nom_provincia" : "$nom_provincia"}, 
                      "longitud" : { "$sum" : "$longitud"}}},    
        {"$sort" : {"longitud" : -1}},
        # {"$limit" : 10},
        {"$project" : {"_id"             : 0, 
                       "comunidad"       : "$_id.comunidad", 
                       "nom_provincia"   : "$_id.nom_provincia", 
                       "metros_de_playa" : "$longitud"}
        }
    ])

In [23]:
df = pd.DataFrame(list(rs))
df

,comunidad,metros_de_playa,nom_provincia
0,Galicia,157040,A Coruña
1,Galicia,93552,Pontevedra
2,Galicia,27217,Lugo


### QUERY_6) ¿ Qué Comunidad tiene más metros de playa ?

In [24]:
rs = db_MDB.playas.aggregate([
        {"$group" : { "_id" : "$comunidad",
                      "longitud" : { "$sum" : "$longitud"}}},    
        {"$sort" : {"longitud" : -1}},
        #  {"$limit" : 10},
        {"$project" : {"_id"             : 0, 
                       "comunidad"       : "$_id", 
                       "metros_de_playa" : "$longitud" }
        }
    ])

In [25]:
df = pd.DataFrame(list(rs))
df

,comunidad,metros_de_playa
0,Andalucía,545459
1,Galicia,277809
2,Comunitat Valenciana,276872
3,Cataluña/Catalunya,237274
4,Canarias,209374
5,Illes Balears,117767
6,"Murcia, Región de",102431
7,"Asturias, Principado de",70428
8,Cantabria,49880
9,País Vasco/Euskadi,30715


### QUERY_7) ¿ Qué provincia tiene más playas ... ?

#### QUERY_7.1) ¿ Qué provincia tiene más playas ... en número ?

In [26]:
rs = db_MDB.playas.aggregate([
        {"$group" : { "_id" : "$nom_provincia", 
                      "contador":{"$sum":1}}},        
        {"$sort" : {"contador" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id"    : 0, 
                       "nom_provincia"   : "$_id", 
                       "num_playas" : "$contador"}
        }
    ])

type(rs)

pymongo.command_cursor.CommandCursor

In [27]:
df = pd.DataFrame(list(rs))
df

,nom_provincia,num_playas
0,A Coruña,410
1,Pontevedra,370
2,Illes Balears,346
3,Las Palmas,322
4,Santa Cruz de Tenerife,257
5,Asturias,205
6,Murcia,199
7,Girona,183
8,Alicante/Alacant,173
9,Málaga,131


#### QUERY_7.2) ¿ Qué provincia tiene más playas ... con nombres diferentes ?

Este es un caso especial de agregación, ya que cuenta los distintos. Dentro del flujo de la función se realizan varios pasos para resolver esta query:

- Formar un conjunto con los distintos nombres de playas de cada provincia
- Desagregar el conjunto
- Agrupar las provincias, contanto los distintos elementos de cada conjunto

In [28]:
rs = db_MDB.playas.aggregate([
        {"$group"   : { "_id" : "$nom_provincia", "aux_playas" : { "$addToSet" : "$nom_playa"}}},
        {"$unwind"  : "$aux_playas"},
        {"$group"   : { "_id" : "$_id", "cuenta" : { "$sum" : 1}}},
        {"$sort"    : {"cuenta" : -1}},
        {"$project" : {"_id" : 0, "nom_provincia": "$_id", "cuenta": 1}},
        {"$match"   : {"nom_provincia": { "$exists": True, "$ne": None }}},
        {"$limit"   : 10}
    ])

In [29]:
df = pd.DataFrame(list(rs))
df

,cuenta,nom_provincia
0,388,A Coruña
1,350,Pontevedra
2,330,Illes Balears
3,303,Las Palmas
4,228,Santa Cruz de Tenerife
5,199,Asturias
6,193,Murcia
7,179,Girona
8,165,Alicante/Alacant
9,126,Málaga


Truco ...

En este tipo de querys complicadas podemos ver los resultados parciales ...

In [30]:
rs = db_MDB.playas.aggregate([
        {"$group"   : { "_id" : "$nom_provincia", "aux_playas" : { "$addToSet" : "$nom_playa"}}},
        {"$unwind"  : "$aux_playas"},
    ])

pd.DataFrame(list(rs)).head(10)

,_id,aux_playas
0,Melilla,Hipódromo
1,Melilla,Los Cárabos
2,Melilla,La Hípica
3,Melilla,Horcas Coloradas
4,Melilla,San Lorenzo
5,Melilla,Trápana
6,Melilla,Ensenada de los Galápagos
7,Melilla,Alcazaba
8,Granada,Melicena
9,Granada,El Gaiterillo


#### QUERY_7.3)  ¿ Qué provincia tiene más playas ... nudistas ?

In [31]:
rs = db_MDB.playas.aggregate([
        {"$match"   : {"nudista": u"Sí"}},
        {"$group" : { "_id" : "$nom_provincia", 
                      "contador":{"$sum":1}}},        
        {"$sort" : {"contador" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id"    : 0, 
                       "nom_provincia"             : "$_id", 
                       "num_playas_nudistas_puras" : "$contador"}
        }
    ])

type(rs)

pymongo.command_cursor.CommandCursor

In [32]:
df = pd.DataFrame(list(rs))
df

,nom_provincia,num_playas_nudistas_puras
0,Las Palmas,146
1,Illes Balears,79
2,Santa Cruz de Tenerife,27
3,Alicante/Alacant,18
4,A Coruña,17
5,Girona,16
6,Gipuzkoa,16
7,Pontevedra,15
8,Tarragona,9
9,Barcelona,8


#### QUERY_7.4)  ¿ Qué provincia tiene más playas ... con bandera azul ?

In [33]:
rs = db_MDB.playas.aggregate([
        {"$match"   : {"bandera_azul": u"Sí"}},
        {"$group" : { "_id" : "$nom_provincia", 
                      "contador":{"$sum":1}}},        
        {"$sort" : {"contador" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id"    : 0, 
                       "nom_provincia"           : "$_id", 
                       "num_playas_bandera_azul" : "$contador"}
        }
    ])

type(rs)

pymongo.command_cursor.CommandCursor

In [34]:
df = pd.DataFrame(list(rs))
df

,nom_provincia,num_playas_bandera_azul
0,Alicante/Alacant,62
1,Pontevedra,57
2,A Coruña,45
3,Tarragona,43
4,Illes Balears,41
5,Murcia,40
6,Castellón/Castelló,33
7,Girona,31
8,Valencia/València,30
9,Barcelona,29


# FIN